<a href="https://colab.research.google.com/github/masterlyj/self-LLM_Agent_RL/blob/main/4_1_bleu_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 大语言模型策略的应用与实现：翻译质量评估 (BLEU vs chrF)

**4.1 - BLEU、ROUGE 和 N-Grams**

**本节目标**：

1. 比较开源模型 (**NLLB-200**) 与商业接口 (**Google Translate**) 的翻译质量。
2. 掌握在中文语境下正确使用 **BLEU** 指标的方法（解决分词问题）。
3. 引入 **chrF** 指标作为字符级评估的补充。

---

### 1. 环境配置

安装本项目所需的依赖库，包括 Hugging Face 的核心库、中文分词工具 `jieba` 以及评估所需的 `sacrebleu`。

In [ ]:
# 安装必要的库
# transformers: 加载模型
# evaluate & sacrebleu: 评估指标
# jieba: 中文分词
# deep-translator: 调用 Google 翻译接口
!pip install -q transformers evaluate sacrebleu jieba deep-translator sentencepiece protobuf

### 2. 准备数据

我们将定义待翻译的英文源句子，以及人工校对的中文参考译文。参考译文将作为评估质量的“金标准”。

In [ ]:
# 待翻译的英文句子
sentences = [
    "In the previous chapters, you've mainly seen how to work with OpenAI models, and you've had a very practical introduction to Hugging Face's open-source models, the use of embeddings, vector databases, and agents.",
    "These have been very practical chapters in which I've tried to gradually introduce concepts that have allowed you, or at least I hope so, to scale up your knowledge and start creating projects using the current technology stack of large language models."
]

In [ ]:
# 中文参考译文 (Ground Truth)
# 注意：BLEU 要求参考译文格式为列表的列表 [[Ref1], [Ref2]]
reference_translations = [
    ["在前几章中，你主要了解了如何使用OpenAI模型，并且还实际接触了Hugging Face的开源模型、嵌入的使用、向量数据库以及智能体。"],
    ["这些章节都非常实用，在其中我尝试逐步介绍一些概念，希望这些概念能帮助你拓展知识，并开始使用当前的大型语言模型技术栈来创建项目。"]
]

### 3. 模型推理：NLLB-200 (开源模型)

首先使用 Meta 的 **NLLB-200-distilled-600M** 模型进行翻译。这是一个轻量级的多语言翻译模型，适合在 Colab 的 CPU 环境下运行。

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# 加载模型和分词器
model_id = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# 创建翻译管道
translator = pipeline('translation', model=model, tokenizer=tokenizer,
                      src_lang="eng_Latn", tgt_lang="zho_Hans")

# 执行翻译
translations_nllb = []
print("=== NLLB 翻译过程 ===")
for text in sentences:
    output = translator(text)
    translated_text = output[0]['translation_text']
    translations_nllb.append(translated_text)
    print(f"原文: {text}...")
    print(f"译文: {translated_text}\n")

### 4. 模型推理：Google Translate (商业基准)

作为对比，我们调用 Google Translate API，代表目前通用的高水平机器翻译质量。

In [ ]:
from deep_translator import GoogleTranslator

# 初始化转换器
translator_google = GoogleTranslator(source='auto', target='zh-CN')

# 执行翻译
translations_google = []
print("=== Google 翻译过程 ===")
for text in sentences:
    translated_text = translator_google.translate(text)
    translations_google.append(translated_text)
    print(f"原文: {text[:50]}...")
    print(f"译文: {translated_text}\n")

### 5. 质量评估：BLEU 指标 (词级别)

**注意**：标准的 BLEU 算法依赖空格来切分单词。中文文本如果不进行分词处理，BLEU 算法将无法识别 n-gram 重叠，导致分数为 0。

为了解决这个问题，我们需要引入 **jieba** 库对中文进行分词预处理。

In [ ]:
import evaluate
import jieba

# 定义中文分词预处理函数
def process_chinese_text(text):
    # 将 "我爱编程" 转换为 "我 爱 编程"
    return " ".join(jieba.cut(text))

# 1. 对预测结果进行分词处理
nllb_tokenized = [process_chinese_text(text) for text in translations_nllb]
google_tokenized = [process_chinese_text(text) for text in translations_google]

# 2. 对参考译文进行分词处理 (注意保持嵌套结构)
refs_tokenized = [[process_chinese_text(ref) for ref in ref_list] for ref_list in reference_translations]

# 3. 计算 BLEU
bleu = evaluate.load("bleu")

score_nllb_bleu = bleu.compute(predictions=nllb_tokenized, references=refs_tokenized)
score_google_bleu = bleu.compute(predictions=google_tokenized, references=refs_tokenized)

print(f"=== 基于 Jieba 分词的 BLEU 评估结果 ===")
print(f"NLLB-200: {score_nllb_bleu['bleu']:.4f}")
print(f"Google:   {score_google_bleu['bleu']:.4f}")

### 6. 进阶评估：chrF 指标 (字符级别)

对于像中文这样不以空格分隔的语言，**chrF (Character n-gram F-score)** 通常被认为是比 BLEU 更稳健的指标。它直接在字符级别计算重叠，不需要依赖外部的分词工具。

In [ ]:
# 加载 chrF 指标
chrf = evaluate.load("chrf")

# chrF 可以直接处理原始字符串，无需分词
score_nllb_chrf = chrf.compute(predictions=translations_nllb, references=reference_translations)
score_google_chrf = chrf.compute(predictions=translations_google, references=reference_translations)

print(f"=== chrF 评估结果 (无需分词) ===")
print(f"NLLB-200: {score_nllb_chrf['score']:.2f}")
print(f"Google:   {score_google_chrf['score']:.2f}")

### 7. 结论与可视化

通过对比两种指标，我们可以得出以下结论。

In [ ]:
import pandas as pd
from IPython.display import display

# 汇总数据
data = {
    "Model": ["NLLB-200 (600M)", "Google Translate"],
    "BLEU Score (Word-level)": [score_nllb_bleu['bleu'], score_google_bleu['bleu']],
    "chrF Score (Char-level)": [score_nllb_chrf['score'], score_google_chrf['score']]
}

df = pd.DataFrame(data)

# 简单的文本可视化
print("=== 最终评估报告 ===")
display(df)

winner = "Google Translate" if score_google_chrf['score'] > score_nllb_chrf['score'] else "NLLB"
print(f"\n结论: 在本测试集中，{winner} 的翻译质量显著优于对比模型。")
print("原因分析: NLLB-200-distilled 是轻量级蒸馏模型，虽然速度快，但在专业术语(如'Embeddings'翻译为'嵌入式')的处理上不如大规模商业模型准确。")